# Push Docker Image to AWS Elastic Container Registry

In [1]:
!cd container; ./build_and_push.sh drug-prediction-gcn

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  37.89kB
Step 1/13 : ARG REGION=us-west-2
Step 2/13 : FROM continuumio/miniconda3
latest: Pulling from continuumio/miniconda3

62c62ec6: Pulling fs layer 
c0154f16: Pulling fs layer 
Digest: sha256:6c979670684d970f8ba934bf9b7bf42e77c30a22eb96af1f30a039b484719159
Status: Downloaded newer image for continuumio/miniconda3:latest
 ---> 406f2b43ea59
Step 3/13 : RUN apt-get update
 ---> Running in ac74f5b46078
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]
Get:3 http://security.debian.org/debian-security buster/updates/main amd64 Packages [174 kB]


# Run Local Training Job

In [1]:
# Download the dataset to local

# download pretrained mol2vec model
!wget -O data/model_300dim.pkl https://github.com/samoturk/mol2vec/raw/master/examples/models/model_300dim.pkl model_300dim.pkl   

# download example data     
!wget -O data/ames.sdf https://github.com/samoturk/mol2vec/raw/master/examples/data/ames.sdf
    
#download hiv data
!wget -O data/hiv.zip https://s3-us-west-1.amazonaws.com/deepchem.io/datasets/molnet_publish/hiv.zip
!cd data; unzip hiv.zip

--2020-01-21 19:23:11--  https://github.com/samoturk/mol2vec/raw/master/examples/models/model_300dim.pkl
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/samoturk/mol2vec/master/examples/models/model_300dim.pkl [following]
--2020-01-21 19:23:12--  https://raw.githubusercontent.com/samoturk/mol2vec/master/examples/models/model_300dim.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76786055 (73M) [application/octet-stream]
Saving to: ‘data/model_300dim.pkl’

data/model_300dim.p 100%[===================>]  73.23M  42.8MB/s    in 1.7s    

2020-01-21 19:23:15 (42.8 MB/s) - ‘data/model_300dim.pkl’ saved [76786055/76786055]

--2020-01

In [2]:
from sagemaker import get_execution_role

role = get_execution_role()

In [3]:
import os
import subprocess

instance_type = 'local'

if subprocess.call('nvidia-smi') == 0:
    ## Set type to GPU if one is present
    instance_type = 'local_gpu'
    
print("Instance type = " + instance_type)

Instance type = local


In [4]:
from sagemaker.estimator import Estimator

hyperparameters = {'dev-mode':True}

estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name='drug-prediction-gcn:latest',
                      hyperparameters=hyperparameters)

estimator.fit('file:///home/ec2-user/SageMaker/graph_neural_net/data')

Creating tmp2zh87u6__algo-1-0svqp_1 ... 
Attaching to tmp2zh87u6__algo-1-0svqp_12mdone
algo-1-0svqp_1  | 2020-01-26 21:52:01,629 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-0svqp_1  | 2020-01-26 21:52:01,629 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value True to Json.
algo-1-0svqp_1  | Returning the value itself
algo-1-0svqp_1  | 2020-01-26 21:52:01,646 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value True to Json.
algo-1-0svqp_1  | Returning the value itself
algo-1-0svqp_1  | 2020-01-26 21:52:01,648 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-0svqp_1  | 2020-01-26 21:52:01,661 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value True to Json.
algo-1-0svqp_1  | Returning the value itself
algo-1-0svqp_1  | 2020-01-26 21:52:01,663 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-0svqp_1  | 2020-01

# Run A Sagemaker Training Job

In [61]:
import sagemaker as sage

sess = sage.Session()

In [62]:
import boto3

client = boto3.client('sts')
account = client.get_caller_identity()['Account']

my_session = boto3.session.Session()
region = my_session.region_name

algorithm_name = 'drug-prediction-gcn'

ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

print(ecr_image)

405787280505.dkr.ecr.us-west-2.amazonaws.com/drug-prediction-gnn:latest


In [63]:
from sagemaker.estimator import Estimator

data_location = 's3://jdurago-insight-2020a/data'
output_path = 's3://jdurago-insight-2020a/output/baseline/'
max_run_time = 3*60*60 # train for max of 3 hours
hyperparameters = {'dev-mode': False}

# instance_type = 'ml.p2.xlarge'
instance_type = 'ml.m5.4xlarge'
# instance_type = 'ml.m5.xlarge'

estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name=ecr_image,
                      hyperparameters=hyperparameters,
                     output_path = output_path,
                     train_max_run=max_run_time)

estimator.fit(data_location)


2020-01-24 23:33:03 Starting - Starting the training job...
2020-01-24 23:33:05 Starting - Launching requested ML instances......
2020-01-24 23:34:05 Starting - Preparing the instances for training...
2020-01-24 23:34:52 Downloading - Downloading input data
2020-01-24 23:34:52 Training - Downloading the training image...............
2020-01-24 23:37:22 Training - Training image download completed. Training in progress.2020-01-24 23:37:22,704 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-01-24 23:37:22,705 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value False to Json.
Returning the value itself
2020-01-24 23:37:23,363 sagemaker-containers INFO     Failed to parse hyperparameter dev-mode value False to Json.
Returning the value itself
2020-01-24 23:37:23,366 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-01-24 23:37:23,376 sagemaker-containers INFO     Failed to parse hyperparameter dev-mod